In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .enableHiveSupport() \
    .getOrCreate()

## 10.3 스파크 SQL
+ 스파크 SQL은 하이브 메타스토어를 사용하므로 하이브와 잘 연동됨
    + 하이브 버전 설정: spark.sql.hive.metastore.version
    + 초기화 방식 변경: spark.sql.hive.metastore.jars
    + 클래스 접두사 설정: spark.sql.hive.metastore.sharedPrefixes

## 10.4 스파크 쿼리 실행방법
+ 스파크 SQL CLI
    ```
    ./bin/spark-sql
    ```
+ 스파크 프로그래밍 SQL 인터페이스
    ```
    spark.sql("SELECT 1+1").show()
    ```
+ 스파크 SQL 쓰리프트 JDBC/ODBC 서버
    + 자바 데이터베이스 연결 인터페이스 제공
    + 하이브 1.2.1. 버전의 HiveServer2에 맞추어 구현

In [2]:
### 프로그래밍 SQL 인터페이스
spark.read.json("./data/flight-data/json/2015-summary.json").createOrReplaceTempView("flight")

spark.sql(
"""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM flight GROUP BY DEST_COUNTRY_NAME
""").show()

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|            Anguilla|        41|
|              Russia|       176|
|            Paraguay|        60|
|             Senegal|        40|
|              Sweden|       118|
|            Kiribati|        26|
|              Guyana|        64|
|         Philippines|       134|
|            Djibouti|         1|
|            Malaysia|         2|
|           Singapore|         3|
|                Fiji|        24|
|              Turkey|       138|
|                Iraq|         1|
|             Germany|      1468|
|              Jordan|        44|
|               Palau|        30|
|Turks and Caicos ...|       230|
|              France|       935|
|              Greece|        30|
+--------------------+----------+
only showing top 20 rows



## 10.5 카탈로그
+ 스파크의 가장 높은 추상화 단계
+ 테이블, 데이터베이스, 함수를 조회하는 등 여러 가지 유용한 함수를 제공

## 10.6 테이블
+ DataFrame과 논리적으로 동일
+ 스파크에서 테이블을 생성하면 default 데이터베이스에 등록됨
+ 임시 테이블 개념이 없으며 데이터를 가지지 않은 뷰만 존재
+ 관리형 테이블과 외부 테이블이 있으며 디스크에 저장된 파일을 이용해 테이블을 정의하면 외부 테이블을 활용하는 것임

### 10.6.2 테이블 생성하기
+ USING을 통해 포맷을 지정하지 않으면 스파크는 기본적으로 하이브 SerDe 설정을 사용해서 Reader, Writer 성능에 악영향을 줌

In [3]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS flights(
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path './data/flight-data/json/2015-summary.json')
""")

DataFrame[]

In [10]:
# 해당 폴더를 지우는 명령어
! rm -r -f ./spark-warehouse/flights_from_select

# 특정 테이블에서 원하는 데이터만 추출해서 새로운 테이블을 생성(CTAS 패턴)
spark.sql( 
"""
CREATE TABLE IF NOT EXISTS flights_from_select
USING Parquet AS SELECT * FROM flights
""") # spark-warehouse 폴더에 테이블 이름의 서브폴더가 생성되고 그 안에 파케이 파일이 저장됨

DataFrame[]

### 10.6.3 외부 테이블 생성하기
+ 외부 테이블의 메타데이터를 관리하지만 데이터 파일은 스파크에서 관리하지 않음

In [5]:
spark.sql(
"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    hive_flights(DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION './data/flight-data-hive'
""")

++
||
++
++



In [6]:
spark.sql(
"""
CREATE EXTERNAL TABLE hive_flights2
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION './data/flight-data-hive' AS SELECT * FROM flights
""")

DataFrame[]

### 10.6.4 테이블에 데이터 삽입하기

In [34]:
spark.sql(
"""
SELECT count(DEST_COUNTRY_NAME) FROM flights_from_select
""").show()

spark.sql(
"""
INSERT INTO flights_from_select
    SELECT * FROM flights LIMIT 20
""")

spark.sql(
"""
SELECT count(DEST_COUNTRY_NAME) FROM flights_from_select
""").show()

+------------------------+
|count(DEST_COUNTRY_NAME)|
+------------------------+
|                     380|
+------------------------+

+------------------------+
|count(DEST_COUNTRY_NAME)|
+------------------------+
|                     400|
+------------------------+



In [27]:
spark.sql( # 메타 데이터 확인
"""
DESCRIBE TABLE flights
""").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   null|
|ORIGIN_COUNTRY_NAME|   string|   null|
|              count|   bigint|   null|
+-------------------+---------+-------+



### 기타 명령어
+ DESCRIBE: 메타정보 확인
+ SHOW PARTITIONS: 파티셔닝 스키마 정보 확인
+ REFRESH: 캐싱된 항목을 갱신
+ MSCK REPAIR TABLE: 관리하는 테이블의 파티션 정보를 새로 고침
+ DROP TABLE: 외부 테이블을 제거하면 데이터는 삭제되지 않지마, 외부 테이블명을 이용해 데이터 조회는 안됨
+ CACHE/UNCACHE TABLE

## 10.10 고급 주제
+ 구조체, 리스트, 맵 세가지 타입이 존재

#### 리스트
+ 값의 리스트를 만드는 collect_list, 중복값을 제거하는 collect_set이 있음

In [62]:
sql = """
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts,
    collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME
"""
spark.sql(sql).show()

+--------------------+-------------+---------------+
|            new_name|flight_counts|     origin_set|
+--------------------+-------------+---------------+
|            Anguilla|         [41]|[United States]|
|            Paraguay|         [60]|[United States]|
|              Russia|        [176]|[United States]|
|             Senegal|         [40]|[United States]|
|              Sweden|        [118]|[United States]|
|            Kiribati|         [26]|[United States]|
|              Guyana|         [64]|[United States]|
|         Philippines|        [134]|[United States]|
|            Djibouti|          [1]|[United States]|
|            Malaysia|          [2]|[United States]|
|           Singapore|          [3]|[United States]|
|                Fiji|         [24]|[United States]|
|              Turkey|        [138]|[United States]|
|                Iraq|          [1]|[United States]|
|             Germany|       [1468]|[United States]|
|              Jordan|         [44]|[United St

In [36]:
sql = """
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights
""" # 직접 베열 입력
spark.sql(sql).show()

+--------------------+--------------+
|   DEST_COUNTRY_NAME|array(1, 2, 3)|
+--------------------+--------------+
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|               Egypt|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|          Costa Rica|     [1, 2, 3]|
|             Senegal|     [1, 2, 3]|
|             Moldova|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|              Guyana|     [1, 2, 3]|
|               Malta|     [1, 2, 3]|
|            Anguilla|     [1, 2, 3]|
|             Bolivia|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
|             Algeria|     [1, 2, 3]|
|Turks and Caicos ...|     [1, 2, 3]|
|       United States|     [1, 2, 3]|
+--------------------+--------------+
only showing top 20 rows



In [37]:
sql = """
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3)[0] FROM flights 
""" # 위치 인덱싱
spark.sql(sql).show()

+--------------------+-----------------+
|   DEST_COUNTRY_NAME|array(1, 2, 3)[0]|
+--------------------+-----------------+
|       United States|                1|
|       United States|                1|
|       United States|                1|
|               Egypt|                1|
|       United States|                1|
|       United States|                1|
|       United States|                1|
|          Costa Rica|                1|
|             Senegal|                1|
|             Moldova|                1|
|       United States|                1|
|       United States|                1|
|              Guyana|                1|
|               Malta|                1|
|            Anguilla|                1|
|             Bolivia|                1|
|       United States|                1|
|             Algeria|                1|
|Turks and Caicos ...|                1|
|       United States|                1|
+--------------------+-----------------+
only showing top

In [52]:
# collect_list로 count 값들을 묶음 
sql = """
CREATE OR REPLACE TEMP VIEW flights_agg AS
    SELECT DEST_COUNTRY_NAME, collect_list(count) as flight_counts
    FROM flights GROUP BY DEST_COUNTRY_NAME
"""
spark.sql(sql).show()

++
||
++
++



In [63]:
# 반대로 동작
sql = """
SELECT explode(flight_counts) as count, DEST_COUNTRY_NAME FROM flights_agg
"""
spark.sql(sql).show()

+-----+--------------------+
|count|   DEST_COUNTRY_NAME|
+-----+--------------------+
|   41|            Anguilla|
|   60|            Paraguay|
|  176|              Russia|
|   40|             Senegal|
|  118|              Sweden|
|   26|            Kiribati|
|   64|              Guyana|
|  134|         Philippines|
|    1|            Djibouti|
|    2|            Malaysia|
|    3|           Singapore|
|   24|                Fiji|
|  138|              Turkey|
|    1|                Iraq|
| 1468|             Germany|
|   44|              Jordan|
|   30|               Palau|
|  935|              France|
|  230|Turks and Caicos ...|
|   30|              Greece|
+-----+--------------------+
only showing top 20 rows



### 10.10.2 함수
+ SHOW FUNCTIONS
+ SHOW SYSTEM FUNCTIONS
+ SHOW USER FUNCTIONS
+ SHOW FUNCTIONS "S*"
+ SHOW FUNCTIONS LIKE "collect*"
+ 사용자 정의 함수

In [67]:
def power3(num):
    return num * num * num

spark.udf.register("power3", f=power3)

sql = "SELECT count, power3(count) FROM flight"
spark.sql(sql).show()

+-----+-------------+
|count|power3(count)|
+-----+-------------+
|   15|         3375|
|    1|            1|
|  344|     40707584|
|   15|         3375|
|   62|       238328|
|    1|            1|
|   62|       238328|
|  588|    203297472|
|   40|        64000|
|    1|            1|
|  325|     34328125|
|   39|        59319|
|   64|       262144|
|    1|            1|
|   41|        68921|
|   30|        27000|
|    6|          216|
|    4|           64|
|  230|     12167000|
|    1|            1|
+-----+-------------+
only showing top 20 rows



### 10.10.3 서브 쿼리
+ 비상호 연관쿼리: 서브쿼리와 연관된 정보를 사용하지 않음
+ 상호 연관쿼리: 내부쿼리가 외부 쿼리의 결과를 참조

#### 비상호 연관쿼리

In [69]:
subsql = """
SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
"""
spark.sql(subsql).show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [70]:
sql = """
SELECT * FROM flights
WHERE origin_country_name IN
    (SELECT dest_country_name FROM flights
    GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
"""
spark.sql(sql).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Egypt|      United States|   15|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|Saint Vincent and...|      United States|    1|
|               Italy|      United States|  382|
|            Pakistan|      United States|   12|
|             Iceland|      United States|  181|
|    Marshall Islands|      United States|   42|
|          Luxembourg|      United States|  155|
|            Honduras|      United States|  362|
|         The Bahama

#### 상호 연관쿼리
목적기 국가에서 되돌아올 수 있는 항공편이 있는지 알고 싶다면 목적지 국가를 출발지 국가로, 출발지 국가를 목적기 국가로 설정하여 항공편이 있는지 확인

In [72]:
sql = """
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
    WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
    WHERE f2.dest_country_name = f1.origin_country_name)
"""
spark.sql(sql).show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|               Malta|       United States|    1|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|Turks and Caicos ...|       United States|  230|
